# 快速开始

本节介绍机器学习中常见任务的 API。请参阅每个部分中的链接以进行更深入的研究。

# 处理数据

PyTorch 有两个[处理数据的基元](https://pytorch.org/docs/stable/data.html)： `torch.utils.data.DataLoader`和`torch.utils.data.Dataset`。 `Dataset`存储样本及其相应的标签，`DataLoader` 包装一个可迭代对象 `Dataset`。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供特定领域的库，例如[TorchText](https://pytorch.org/text/stable/index.html)、 [TorchVision](https://pytorch.org/vision/stable/index.html)和[TorchAudio](https://pytorch.org/audio/stable/index.html)，所有这些都包含数据集。在本教程中，我们将使用 TorchVision 数据集。

该`torchvision.datasets`模块包含`Dataset`许多现实世界视觉数据的对象，如 CIFAR、COCO（[完整列表见此处](https://pytorch.org/vision/stable/datasets.html)）。在本教程中，我们使用 FashionMNIST 数据集。每个 TorchVision 都`Dataset`包含两个参数：`transform`和 `target_transform`分别修改样本和标签。

In [2]:
# Download training data from open datasets.
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将 `Dataset` 作为参数传递给 `DataLoader`。这在我们的数据集上包装了一个可迭代对象，and supports automatic batching, sampling, shuffling and multiprocess data loading.这里我们定义batch size为64，即每个元素 在数据加载器中，iterable 将返回一批 64 个特征和标签。

In [3]:
batch_size=64

# Create data loaders.
train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W], {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W], torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


详细了解[在 PyTorch 中加载数据](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)。

# 创建模型

为了在 PyTorch 中定义神经网络，我们创建一个继承自[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的类。我们在`__init__`函数中，定义网络的层。在`forward`函数中，指定数据如何在函数中通过网络。为了加速神经网络中的操作，我们将其转移到 GPU 或 MPS（如果可用）。

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
    
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


阅读有关[在 PyTorch 中构建神经网络](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)的更多信息。

# 优化模型参数

为了训练模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions) 和一个[优化器](https://pytorch.org/docs/stable/optim.html)。

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

在单个训练循环中，模型对训练数据集（批量输入），进行预测。并反向传播预测误差，以调整模型的参数。

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

我们还根据测试数据集检查模型的性能，以确保它正在学习。

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需要经过多次迭代（*epochs*）。在每个时期，模型都会学习参数以做出更好的预测。我们打印每个时期模型的准确性和损失；我们希望看到每个 epoch 的准确率都会提高，损失也会减少。

In [8]:
epochs = 5
for t in range (epochs):
    print(f"Epoch {t+1}\n--------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 2.292588 [   64 / 60000]
loss: 2.281483 [ 6464 / 60000]
loss: 2.263758 [12864 / 60000]
loss: 2.261707 [19264 / 60000]
loss: 2.238106 [25664 / 60000]
loss: 2.213899 [32064 / 60000]
loss: 2.212728 [38464 / 60000]
loss: 2.178851 [44864 / 60000]
loss: 2.174153 [51264 / 60000]
loss: 2.145497 [57664 / 60000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 0.013894 

Epoch 2
--------------------
loss: 2.147638 [   64 / 60000]
loss: 2.131939 [ 6464 / 60000]
loss: 2.077182 [12864 / 60000]
loss: 2.093227 [19264 / 60000]
loss: 2.033344 [25664 / 60000]
loss: 1.976775 [32064 / 60000]
loss: 1.992444 [38464 / 60000]
loss: 1.911830 [44864 / 60000]
loss: 1.923178 [51264 / 60000]
loss: 1.840364 [57664 / 60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 0.012120 

Epoch 3
--------------------
loss: 1.888993 [   64 / 60000]
loss: 1.844798 [ 6464 / 60000]
loss: 1.739015 [12864 / 60000]
loss: 1.775688 [19264 / 60000]
loss: 1.662096 [25664 / 60000]
loss: 1.625208 [32064 / 60000]
los

# 保存模型

保存模型的常见方法是序列化内部状态字典（包含模型参数）。

In [9]:
torch.save(model.state_dict(),"model.pth")
print("Save PyTorch Model State to model.pth")

Save PyTorch Model State to model.pth


# 加载模型

加载模型的过程包括：重新创建模型结构，并将状态字典加载到其中。

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

该模型现在可用于进行预测。

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
